In [1]:
!pip install unsloth wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 62.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 10.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 98.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.8 MB/s eta 0:00:0

## Unsloth

In [4]:
from kaggle_secrets import UserSecretsClient
secrets_client = UserSecretsClient()

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

qwen_models = [
    "unsloth/Qwen2.5-3B-Instruct-unsloth-bnb-4bit",
    "unsloth/Qwen2.5-7B-Instruct-unsloth-bnb-4bit", # unsloth-bnb-4bit are selectively quantized for more accuracy
    "unsloth/Qwen2.5-3B-Instruct-bnb-4bit", # not selectively quantized
] # More models at https://huggingface.co/unsloth

MODEL = qwen_models[1]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = secrets_client.get_secret("HF_TOKEN")
)

==((====))==  Unsloth 2025.5.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Add LoRA adaptors

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Data prep

In [6]:
from unsloth.chat_templates import get_chat_template

SYSTEM_PROMPT = """
You are Choreo Assistant, an AI trained specifically to help users understand and work with Choreo — the integration platform as a service (iPaaS) developed by WSO2.

Your role is to provide accurate, concise, and actionable answers based strictly on the Choreo documentation you were fine-tuned on. You assist users in performing tasks, understanding features, troubleshooting issues, and navigating workflows within the Choreo platform.

When responding:
- Focus only on Choreo-related topics such as webhooks, services, APIs, integrations, authentication, CI/CD, external consumers, observability, and other Choreo platform capabilities.
- Avoid speculation or hallucination — if you're unsure or the information is unavailable, say: *"I'm not certain about that. Please refer to the official Choreo documentation for the latest guidance."*
- Provide answers in a clear, easy-to-follow format, ideally using step-by-step instructions or bullet points where helpful.
- When relevant, provide links to specific sections of the official documentation.

You must *not* respond to requests unrelated to Choreo (e.g., jokes, general trivia, non-technical queries).

Stay professional, focused, and documentation-aligned at all times.

"""

def convert_to_conversations(example, system_prompt=SYSTEM_PROMPT):
    instruction_part = system_prompt if system_prompt else example['instruction'].strip()
    user_part = example['input'].strip()
    assistant_part = example['output'].strip()

    return {
        "conversations": [
            {"role": "system", "content": instruction_part},
            {"role": "user", "content": user_part},
            {"role": "assistant", "content": assistant_part}
        ]
    }

def formatting_prompts_func(examples):
    convos = examples['conversations']
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("rtweera/choreo-dataset-fixed-leakage", split = "train")

Generating train split:   0%|          | 0/654 [00:00<?, ? examples/s]

In [7]:
dataset = dataset.map(convert_to_conversations, remove_columns=["instruction", "input", "output"], batched=False,) # Non batch operations (like strip()) present

Map:   0%|          | 0/654 [00:00<?, ? examples/s]

In [8]:
dataset = dataset.map(formatting_prompts_func, batched=True,) # inside only batch operations are allowed

Map:   0%|          | 0/654 [00:00<?, ? examples/s]

## Run config

In [9]:
from datetime import datetime
import pytz

MODEL_SAVENAME = MODEL.split('/')[-1]
now_utc = datetime.now(pytz.utc)
now_colombo = now_utc.astimezone(pytz.timezone('Asia/Colombo'))
time_str = now_colombo.strftime('%Y-%b-%d_%H-%M-%S')
run_name = f'{time_str}_{MODEL_SAVENAME}_LoRA'
print(run_name)

2025-May-22_15-43-18_Qwen2.5-7B-Instruct-unsloth-bnb-4bit_LoRA


In [10]:
import wandb
from huggingface_hub import login

In [11]:
login(token=secrets_client.get_secret("HF_TOKEN"))
wandb.login(key=secrets_client.get_secret('WANDB_TOKEN'))
wandb.init(project="choreo-doc-assistant-lora", name=run_name)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rtweera (rtw-rtweera) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Train

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, # Fixed major bug in latest Unsloth
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        logging_first_step=True,
        optim = "paged_adamw_8bit", # Save more memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = run_name,
        report_to = "wandb", # Use this for WandB etc
        save_steps=20,
        save_total_limit=4,
        push_to_hub=True,
        hub_model_id=run_name
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/654 [00:00<?, ? examples/s]

train_on_responses masks all the user input (as well as system prompt) and only calculate the loss on assistant output
[https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen2.5_Coder_(14B)-Conversational.ipynb#scrollTo=juQiExuBG5Bt]

In [13]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=4):   0%|          | 0/654 [00:00<?, ? examples/s]

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 654 | Num Epochs = 3 | Total steps = 489
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.399900
2,1.473300
3,1.490400
4,1.515600
5,1.510400
6,1.215300
7,1.095100
8,1.156900
9,1.155100
10,1.190500


## Check generation

In [15]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "How to deploy a webapp in choreo?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024,
                   use_cache = True, temperature = 1.5, min_p = 0.1) # 

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


To deploy a web application in Choreo, follow these steps:\n\n1. **Create a Web Application**:\n   - Ensure you have created a web application component either from scratch or by starting with a **GitHub** or **Docker Hub** repository.\n\n2. **Select Your Component**:\n   - In the Choreo Console, navigate to the **Component Listing** pane and select the web application for which you want to create a deployment configuration.\n\n3. **Open Deployment Settings**:\n   - Go to the left navigation menu and click on **DevOps**, then select **Deployment Configurations**.\n\n4. **Edit Default Deployment Configuration**:\n   - On the **Deployment Configurations** page, click to edit the default deployment configuration.\n\n5. **Define Build Configurations**:\n   - Fill in the build configurations as needed:\n     | **Field**           | **Values**                                                                                                                                                       

In [16]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What is choreo?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 2048,
                   use_cache = True, temperature = 1.5, min_p = 0.1) # 

Choreo is a cloud-native integration platform as a service (iPaaS). It allows developers to easily create, manage, and monitor API proxies and microservices, enabling seamless communication and integration across various applications and services.\n\nHere are some key aspects of Choreo:\n\n- **Integration Capabilities**: It provides functionalities to connect different systems and services, making it easier to develop comprehensive solutions.\n- **Service Deployment**: Choreo supports deploying various types of services, including REST APIs, GraphQL APIs, gRPC services, and HTTP servers.\n- **CI/CD**: It offers integrated CI/CD pipelines, streamlining the development process.\n- **Observability**: The platform includes robust observability tools to monitor service performance and health, allowing developers to troubleshoot issues effectively.\n- **User-Friendly Interface**: Choreo is designed to be user-friendly, supporting both command-line interfaces (CLI) and a user-friendly web con

## Generate eval set answers

In [17]:
user_prompts = [
    "How to deploy a webapp?",
    "How to resolve \"\"Module not found error\"\" during the deployment of a Python project?",
    "How to add authentication to my service?",
    "How to add a custom domain to my web app?",
    "How can I test my service?",
    "How can I add environment variables to python webapp?",
    "How to connect the React frontend to the backend?",
    "How can I deploy my backend and then connect it to the frontend deployed in github pages?",
    "How can I implement testing for a full-stack application including automated testing for both frontend and backend components?",
    "I'm getting \"\"procfile not found\"\" error for my python service. How do I resolve this?",
    "I'm getting .choreo/endpoints.yaml not found error. How do I resolve this?",
    "Tell me how I can configure a readiness probe?",
    "How to configure Azure as an external IdP?",
    "How can I implement rate limiting on my APIs?",
    "How can i resolve trivy security check errors?",
    "How can I add a config file to react web app?",
    "How can I secure my web app?",
    "What are the steps to secure an API in Choreo?",
    "How can I set up alerts for my service?",
    "How can I add members to my organization",
    "How can I create a new environment for deployment",
    "How to make sure that an api is only visible to admins in my organization?",
    "How to use Choreo CLI locally?",
    "How do I set up automatic scaling for my applications?",
    "What options are available for data storage in Choreo?",
    "How can I create a webhook?",
    "Can I set usage limits for my API based on different tiers",
    "How can I monitor the performance of my services deployed on Choreo?",
    "How can I deploy a new version of my service from a different branch in the same component?",
    "What are the available pricing plans in Choreo?",
    "How can I limit requests coming to my API?",
    "I want to control traffic coming to my API.",
    "My build fails during trivy scan",
    "I get build errors during vulnerability scan stage",
    "I have secret keys as configs for my react web app. How to manage them securely?",
    "What steps can I take to protect my web app?",
    "What methods can I use to protect an API in Choreo?",
    "How do I invite new people to join my organization?",
    "How do I restrict my API so that only admins can see it?",
    "How do I get started with using Choreo CLI on my computer?",
    "How do I configure my applications to scale automatically?",
    "My service gets overloaded with requests how to fix this?"
]

In [18]:
len(user_prompts)

42

In [19]:
from tqdm import tqdm
output_file_path = "output.txt"

gen_kwargs = {
    "max_new_tokens": 2048,
    "use_cache": True,
    "temperature": 1.5,
    "min_p": 0.1,
}

# input_ids = inputs, streamer = text_streamer, max_new_tokens = 2048,
#                    use_cache = True, temperature = 1.5, min_p = 0.1)

with open(output_file_path, "w", encoding="utf-8") as f:
    for prompt in tqdm(user_prompts, desc="Generating"):
        # Create chat format input
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ]
        prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        # Generate
        inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
        output_ids = model.generate(**inputs, **gen_kwargs)
        output_text = tokenizer.decode(output_ids[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

        # Write one output per line
        clean_output = output_text.replace("\n", " ").strip()
        # f.write(output_text.strip().replace("\n", " ") + "\n")
        clean_output = f'"{clean_output}"'
        f.write(clean_output + "\n")

Generating: 100%|██████████| 42/42 [15:12<00:00, 21.73s/it]


## Load model

In [4]:
!pip install unsloth wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 55.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 7.7 MB/s eta 0:00:

In [5]:
from kaggle_secrets import UserSecretsClient
secrets_client = UserSecretsClient()

from unsloth import FastLanguageModel
import torch

LOAD_NAME = "rtweera/2025-May-22_12-10-49_Qwen2.5-3B-Instruct-unsloth-bnb-4bit_LoRA"

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LOAD_NAME, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = secrets_client.get_secret("HF_TOKEN")
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-22 10:01:06.374252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747908066.574147      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747908066.631479      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/120M [00:00<?, ?B/s]

Unsloth 2025.5.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

In [6]:
SYSTEM_PROMPT = """
You are Choreo Assistant, an AI trained specifically to help users understand and work with Choreo — the integration platform as a service (iPaaS) developed by WSO2.

Your role is to provide accurate, concise, and actionable answers based strictly on the Choreo documentation you were fine-tuned on. You assist users in performing tasks, understanding features, troubleshooting issues, and navigating workflows within the Choreo platform.

When responding:
- Focus only on Choreo-related topics such as webhooks, services, APIs, integrations, authentication, CI/CD, external consumers, observability, and other Choreo platform capabilities.
- Avoid speculation or hallucination — if you're unsure or the information is unavailable, say: *"I'm not certain about that. Please refer to the official Choreo documentation for the latest guidance."*
- Provide answers in a clear, easy-to-follow format, ideally using step-by-step instructions or bullet points where helpful.
- When relevant, provide links to specific sections of the official documentation.

You must *not* respond to requests unrelated to Choreo (e.g., jokes, general trivia, non-technical queries).

Stay professional, focused, and documentation-aligned at all times.

"""

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What is choreo?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 2048,
                   use_cache = True, temperature = 1.5, min_p = 0.1) # 

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Choreo is a leading integrated cloud-native development platform designed to streamline the process of developing and managing applications across multiple clouds. It simplifies containerized application development, enabling developers to efficiently create, deploy, test, and manage microservices in a single console. Key features of Choreo include:\n\n1. **Single Console Development**: Choreo provides a unified platform that facilitates building applications, regardless of their deployment locations, making it easier for teams to collaborate and accelerate their development processes.\n\n2. **Full DevOps Workflow**: With Choreo, you can automate tasks like build, deploy, testing, and scaling across various cloud environments, including Azure, AWS, GCP, and Digital Ocean. This includes support for diverse architectures like monolithic, service-oriented, API-first, and full PDP.\n\n3. **Containerization Support**: Choreo supports containerization of applications in various languages and